<h1>Recommender system challenge PoliMi 2018</h1>

In [7]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import recommender as recommender

In [8]:
#see if we upload it correctly
train = pd.read_csv("all/train.csv")
train.head()

,playlist_id,track_id
0,0,14301
1,0,8360
2,0,12844
3,0,18397
4,0,1220


In [9]:
tracks = pd.read_csv("all/tracks.csv")
tracks.head()

,track_id,album_id,artist_id,duration_sec
0,0,6306,449,167
1,1,12085,4903,185
2,2,1885,6358,201
3,3,3989,1150,263
4,4,11633,4447,96


In [10]:
test = pd.read_csv("all/target_playlists.csv")
test.head()

,playlist_id
0,7
1,25
2,29
3,34
4,50


In [11]:
top_rec = recommender.top_pop_rec()
top_rec.create(train, 'playlist_id', 'track_id')

KeyError: 'score'